In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf

In [38]:
symbols = ['AVGO', 'MTD', 'NVR']
for symbol in symbols:
    data = yf.download(symbol, start='2000-01-01', end='2023-02-23')
    data.to_parquet(f'../data/inv_companies/{symbol.lower()}.parquet')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [39]:
MTD = pd.read_parquet('../data/inv_companies/mtd.parquet')
AVGO = pd.read_parquet('../data/inv_companies/avgo.parquet')
NVR = pd.read_parquet('../data/inv_companies/nvr.parquet')

In [50]:
def calc_volatility(df, period):
    # Calcular los cambios porcentuales diarios en el precio de la acción
    daily_returns = df['Close'].pct_change()

    # Calcular la volatilidad según el período especificado
    if period == 'anual':
        volatility = daily_returns.resample('Y').std().iloc[-1] * (252 ** 0.5)
    elif period == 'semestral':
        volatility = daily_returns.resample('6M').std().iloc[-1] * (252 ** 0.5)
    elif period == 'trimestral':
        volatility = daily_returns.resample('Q').std().iloc[-1] * (252 ** 0.5)
    else:
        raise ValueError(f"El período {period} no es válido.")

    # Retornar la volatilidad calculada
    return round(volatility,2)

In [83]:
import pandas as pd

def calculate_cumulative_return(df, period):
    # Selecciona los datos de precios de cierre ajustados para el período de tiempo seleccionado
    df = df.reset_index()
    df_period = df.loc[df['Date'] >= pd.Timestamp.today() - pd.tseries.offsets.DateOffset(months=int(period[:-1])*6)]
    
    # Resample a precios de cierre ajustados mensuales
    df_return = df_period.resample('M', on='Date').last()
    
    # Calcula los retornos mensuales y acumulados
    df_return['return'] = df_return['Adj Close'].pct_change()
    df_return['cumulative_return'] = (1 + df_return['return']).cumprod()
    
    cumulative_return = df_return['cumulative_return'][-1]
    # Imprime la rentabilidad acumulada
    # print(f"Rentabilidad acumulada en los últimos {period}: {cumulative_return}")

    return cumulative_return


In [84]:
calculate_cumulative_return(MTD, '1y')

Rentabilidad acumulada en los últimos 1y: 1.3424805747839428


1.3424805747839428